In [2]:
%matplotlib qt
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.optim as optim

%load_ext autoreload
%autoreload 2

In [3]:
#from forward import *
import sys
sys.path.append("../static/")
sys.path.append("..")
import kinematics
from inverse import with_torch
from presets_robot_models import preset_models
#from learn import evaluate_plot, train_motors
from learn import *

In [4]:
alpha, theta, radius, dists, active, limits = get_DH_params(preset_models["HangingArm"])

In [5]:
For_model = kinematics.Kinematics("HangingArm")

In [166]:
model = torch.load("./kine_models/HangingArm.pt")

# For Training

In [73]:
For_model.orientation = False
inputs, target = For_model.generate_maps()
n_in  = inputs.shape[1]
n_out = target.shape[1]
model = NeuralNetworkStack(n_in, n_out, nlayers=5, depth=20)
optimizer = optim.Adam(model.parameters(), lr=1e-3)

# generate map

In [168]:
inputs, target = For_model.generate_maps()

In [169]:
For_model.orientation = False

In [93]:
evaluate_plot(For_model, model)

tensor(1783.6183, grad_fn=<MeanBackward0>)


IndexError: index 3 is out of bounds for axis 1 with size 3

In [90]:
train_positions(For_model, model)

7379.25
5340.28
4640.36
3938.33
3493.0
2911.0
2815.35
2594.39
2588.31
2677.15
2671.52
2504.1
2530.43
2618.14
2479.56
2417.25
2376.71
2415.34
2153.74
2168.43
2079.71
2067.44
1967.91
2122.92
2083.38
2063.31
2001.52
1996.47
1910.47
2017.83
1882.54
1886.79
1869.01
1983.18
1923.21
1838.77
1843.88
2005.3
1868.58
1985.1
1833.72
1823.41
1929.62
1752.6
1803.12
1849.59
1674.27
1831.73
1788.27
1712.34
1768.89
1796.19
1800.4
1788.81
1784.93
1827.2
1817.2
1817.54
1782.02
1858.07
1758.07
1798.94
1783.52
1723.3
1872.5
1768.98
1869.56
1785.31
1785.22
1905.02
1735.48
1857.2
1863.94
1819.18
1863.56
1746.44
1758.26
1736.46
1812.12
1785.46
1727.15
1745.61
1779.55
1764.02
1865.76
1773.64
1848.99
1868.68
1741.62
1678.8
1742.98
1916.85
1729.51
1843.54
1738.19
1854.94
1733.83
1756.79
1811.82
1854.82
1813.71
1902.4
1840.41
1746.85
1724.62
1741.45
1645.75
1860.03
1829.38
1829.76
1832.06
1735.7
1767.28
1833.38
1701.1
1699.28
1886.81
1852.3
1743.84
1738.18
1755.53
1737.45
1834.24
1765.14
1881.39
1861.27
1739.72
1

KeyboardInterrupt: 

In [89]:
train_motors(For_model, model)

10910.82294921875
8381.51376953125
7840.161474609375
7857.16201171875
7943.798876953125
7542.03603515625
7842.257763671875
7710.9509765625
7956.29150390625
7865.8333984375
7944.95068359375
7806.859912109375
8013.22978515625
7791.94521484375
7430.37158203125
7632.027490234375
7992.9291015625
7594.913037109375
7698.241064453125
8033.4345703125
7627.141845703125
7834.92451171875
8223.026416015626
7752.871435546875
8009.42880859375
7785.232080078125
7593.044775390625
8076.146826171875
8083.0544921875
7857.88330078125
7661.971875
7852.24345703125
7591.3400390625
7544.761669921875
8182.04150390625
7801.605810546875
7757.070849609375
7777.653173828125
7886.44814453125
7845.945849609375
8018.783642578125
7505.630712890625
7984.59404296875
7353.711083984375
7712.79296875
7829.4587890625
7666.9369140625
7723.923046875
8013.70146484375
7572.76669921875
7422.673291015625
8039.899560546875
8034.324169921875
7593.17265625
7369.3806640625
7772.647021484375
7964.13623046875
7736.62802734375
7579.15244

# Blind Testing

In [68]:
For_model.orientation = True
inputs, targets = For_model.generate_paths()
model = NeuralNetworkBlind(inputs.shape[1], targets.shape[1], nlayers=8, depth=10)
optimizer = optim.Adam(model.parameters(), lr=1e-3)

In [39]:
train_positions_paths(For_model, model, 2000)

231.78
192.31  caching model improvement
156.56  caching model improvement
123.23  caching model improvement


KeyboardInterrupt: 

In [46]:
evaluate_plot(For_model, model)

tensor(55484.5938, grad_fn=<MeanBackward0>)


# Prototype code to test Machine

In [ ]:
inputs, target = For_model.generate_maps(n=1)
#mot_pos = np.array([alpha,theta,radius,dists])
n= np.random.randint(len(inputs))
#######################
pred = model(torch.Tensor(inputs[n:n+1])).detach()
######################
For_model.set_active(pred[0])
For_model.update_position()
pos_pred = For_model.run_forward().round(2)
#######################
For_model.set_active(target[n])
pos_targ = For_model.run_forward().round(2)
########################
print("--- x y z ---")
print(inputs[n].round(2))
print(pos_targ)
print(pos_pred )
print("--- torch ---")
pred_xys = with_torch().forward_from_active(For_model, target[n:n+1])
print(pred_xys[0,0].numpy().round(2))
pred_xys = with_torch().forward_from_active(For_model, pred)
print(pred_xys[0,0].numpy().round(2))
print("--- motor positions ---")
print(target[n].round(2))
print(pred.numpy().round(2)[0])

In [ ]:
def get_DH(For_model):
    alpha  = torch.tensor(1.*np.array(For_model.alpha)) 
    theta  = torch.tensor(1.*np.array(For_model.theta)) 
    radius = torch.tensor(1.*np.array(For_model.radius))
    dists  = torch.tensor(1.*np.array(For_model.dists)) 
    active = For_model.active
    DH = torch.stack([alpha, theta, radius, dists])
    return DH

def get_transform(DH):
    alpha, theta, radius, dists = DH
    alpha = torch.deg2rad(alpha)
    theta = torch.deg2rad(theta)
    
    TFs = make_transforms(alpha, theta, radius, dists)
    return TFs[-1]

def make_transforms(alpha, theta, radius, dists):

    mot_params = torch.stack([alpha, theta, radius, dists])

    Tfs = []
    T = torch.eye(4)
    for i in range(mot_params.shape[1]):
        a,t,r,d = mot_params[:,i]
        DHt = DH_transform(a,t,r,d) 
        T = torch.matmul(T,DHt)     
        Tfs.append(T)

    return Tfs

def DH_transform(a,t,r,z):
        Zt = Z_transform(a,t,r,z)
        Xt = X_transform(a,t,r,z)
        DHt = torch.matmul(Zt,Xt)
        return DHt

    ###############################################
def X_transform(a,t,r,d):

    Xt = torch.eye(4)
    Xt[0,3] = r
    Xt[1,1] = torch.cos(a)
    Xt[1,2] = -1*torch.sin(a)
    Xt[2,1] = torch.sin(a)
    Xt[2,2] = torch.cos(a)

    return Xt

def Z_transform(a,t,r,d):

    Zt = torch.eye(4)
    Zt[0,0] = torch.cos(t)
    Zt[0,1] = -1*torch.sin(t)
    Zt[1,0] = torch.sin(t)
    Zt[1,1] = torch.cos(t)
    Zt[2,3] = d

    return Zt

In [ ]:
matrix = torch.Tensor(np.eye(3))

In [ ]:
rot_angle = torch_matrix_to_rotation( matrix )
rot_angle

In [ ]:
DH = get_DH(For_model)
TF = get_transform(DH)
matrix = TF[:3,:3]

In [ ]:
rotation = np.array(rotation)

In [ ]:
DH = get_DH(For_model)
TF = get_transform(DH)
matrix = TF[:3,:3]

matrix = torch.Tensor(R.from_euler("XYZ",[np.pi/2,np.pi/3,np.pi]).as_matrix())

In [ ]:
for _ in range(100):
    rot = np.random.uniform(np.pi,size=(3))
    out = R.from_matrix(R.from_euler("XYZ",rot).as_matrix()).as_euler("XYZ")
    if out[0]<0:  
        out[[0,2]] = out[[0,2]] + np.pi
        out[[1]]   = np.pi - out[[1]] 
    print(rot, out, rot.round(4) == out.round(4)) 

In [ ]:
For_model = kinematics(alpha, theta, radius, dists, active, limits)

In [ ]:
inputs, target = For_model.generate_maps()
#mot_pos = np.array([alpha,theta,radius,dists])
#n= np.random.randint(len(inputs))
pred = model(torch.Tensor(inputs)).detach().numpy()

In [ ]:
For_model = kinematics(alpha, theta, radius, dists, active, limits)

In [ ]:
For_model.orientation = True

## 2D TEST

In [ ]:
size = 1000
t = np.random.randint(-10,370,size=size)
r = np.random.randint(100,size=size)
inputs = np.array([r,t]).T

x = np.sin(np.radians(t))*r
y = np.cos(np.radians(t))*r
target = np.array([x,y]).T

inputs,target = target,inputs

In [ ]:
n_in  = inputs.shape[1]
n_out = target.shape[1]
model = NeuralNetworkStack(n_in, n_out, nlayers=15, depth=100)
#model = NeuralNetworkSinusoid(n_in, n_out)
optimizer = optim.Adam(model.parameters(), lr=1e-3)

In [ ]:
#optimizer = optim.Adam(model.parameters(), lr=1e-3)
for n in range(100):
    size = 500
    t = np.random.randint(-10,370,size=size)
    r = np.random.randint(150,size=size)
    inputs = np.array([r,t]).T

    x = np.sin(np.radians(t))*r
    y = np.cos(np.radians(t))*r
    target = np.array([x,y]).T
    
    train_loop(model, inputs, target, distance_loss,optimizer)
        
    if n % 10 == 0:
        optimizer.param_groups[0]["lr"] = optimizer.param_groups[0]["lr"]*0.5        

In [ ]:
n= np.random.randint(len(inputs))
xy = target[n]
pred = model(torch.Tensor(inputs[n:n+1])).detach().numpy()
print(inputs[n])
print(target[n])
print(pred.round(2))

In [ ]:
size = 1000
t = np.random.randint(-10,370,size=size)
r = np.random.randint(150,size=size)
inputs = np.array([r,t]).T

x = np.sin(np.radians(t))*r
y = np.cos(np.radians(t))*r
target = np.array([x,y]).T

inputs, target = target, inputs

pred = model(torch.Tensor(inputs)).detach().numpy()



r = inputs[:,0]
t = inputs[:,1]

x = target[:,0]
y = target[:,1]

x1 = r*np.sin(np.radians(t))
y1 = r*np.cos(np.radians(t))

x2 = pred[:,0]
y2 = pred[:,1]

x2 = r*np.sin(np.radians(x2))
y2 = r*np.cos(np.radians(y2))

plt.plot(x,y,"bo")
#plt.plot(x1,y1,)
plt.plot(x2,y2,"ro")
plt.plot([x,x2],[y,y2],"g")

In [ ]:
size = 1000

pred = model(torch.Tensor(inputs)).detach().numpy()

x = target[:,0]
y = target[:,1]

x2 = pred[:,0]
y2 = pred[:,1]


plt.plot(x,y,"bo")
#plt.plot(x1,y1,)
plt.plot(x2,y2,"ro")
plt.plot([x,x2],[y,y2],"g")

In [ ]:
    def forward(self,x_in):
                
        x_next = self.layer_first(x_in)
    
        out_list = [x_in,x_next]
        for n, block in enumerate(self.blocks):
            x_next = block(x_next) 
            out_list.append(x_next)  
            
        x = torch.cat(out_list, dim=1)
        
        x = F.leaky_relu(x)
        print(x)

        out = self.layer_last(x)
        
        return out